In [1]:
import boto3
import json

### `Lambda`

In [ ]:
aws_lambda = boto3.client('lambda')
iam_client = boto3.client('iam')
role = iam_client.get_role(RoleName='LabRole')

# TODO: open the zip file containing your Lambda function
with open('<lambda_func_zip>.zip', 'rb') as f:
    lambda_zip = f.read()

In [6]:
try:
    # If function hasn't yet been created, create it
    # TODO: replace the placeholders with your function specifics
    response = aws_lambda.create_function(
        FunctionName='<lambda_function_name>',
        Runtime='python3.11',
        Role=role['Role']['Arn'],
        Handler='<file_name>.<lambda_function_name>',
        Code=dict(ZipFile=lambda_zip),
        Timeout=300
    )
except aws_lambda.exceptions.ResourceConflictException:
    # If function already exists, update it based on zip
    # file contents
    response = aws_lambda.update_function_code(
        FunctionName='<lambda_function_name>',
        ZipFile=lambda_zip
        )

lambda_arn = response['FunctionArn']
lambda_arn

'arn:aws:lambda:us-east-1:591015244613:function:lambda_handler'

In [7]:
# can invoke once serially, or invoke in parallel via Step functions (below)
r = aws_lambda.invoke(FunctionName='<lambda_function_name>',
                      InvocationType='RequestResponse',
                      Payload=json.dumps({'num_points':10000}))
json.loads(r['Payload'].read())

{'statusCode': 200, 'body': {'pi_estimate': 3.1384, 'num_points': 10000}}

### `Step` Functions

In [8]:
sfn = boto3.client('stepfunctions')

def make_def(lambda_arn):
    definition = {
      "Comment": "My State Machine",
      "StartAt": "Map",
      "States": {
        "Map": {
          "Type": "Map",
          "End": True,
          "Iterator": {
            "StartAt": "Lambda Invoke",
            "States": {
              "Lambda Invoke": {
                "Type": "Task",
                "Resource": "arn:aws:states:::lambda:invoke",
                "OutputPath": "$.Payload",
                "Parameters": {
                  "Payload.$": "$",
                  "FunctionName": lambda_arn
                },
                "Retry": [
                  {
                    "ErrorEquals": [
                      "Lambda.ServiceException",
                      "Lambda.AWSLambdaException",
                      "Lambda.SdkClientException",
                      "Lambda.TooManyRequestsException",
                      "States.TaskFailed"
                    ],
                    "IntervalSeconds": 2,
                    "MaxAttempts": 6,
                    "BackoffRate": 2
                  }
                ],
                "End": True
              }
            }
          }
        }
      }
    }
    return definition

In [24]:
sf_def = make_def(lambda_arn)

try:
    response = sfn.create_state_machine(
        name='monte_carlo_sm',
        definition=json.dumps(sf_def),
        roleArn=role['Role']['Arn'],
        type='EXPRESS'
    )
except sfn.exceptions.StateMachineAlreadyExists:
    response = sfn.list_state_machines()
    state_machine_arn = [sm['stateMachineArn'] 
                         for sm in response['stateMachines'] 
                         if sm['name'] == 'monte_carlo_sm'][0]
    response = sfn.update_state_machine(
        stateMachineArn=state_machine_arn,
        definition=json.dumps(sf_def),
        roleArn=role['Role']['Arn']
    )

In [29]:
response = sfn.list_state_machines()
response

{'stateMachines': [{'stateMachineArn': 'arn:aws:states:us-east-1:591015244613:stateMachine:monte_carlo_sm',
   'name': 'monte_carlo_sm',
   'type': 'EXPRESS',
   'creationDate': datetime.datetime(2024, 4, 4, 15, 41, 36, 496000, tzinfo=tzlocal())}],
 'ResponseMetadata': {'RequestId': 'cae953cd-ac02-4754-a371-5feb6c3cb0d8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'cae953cd-ac02-4754-a371-5feb6c3cb0d8',
   'date': 'Thu, 04 Apr 2024 20:59:48 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '180',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [ ]:
state_machine_arn = [sm['stateMachineArn'] 
                     for sm in response['stateMachines'] 
                     if sm['name'] == 'monte_carlo_sm'][0]

# Do not invoke more than 10 workers at a time or you risk your AWS Academy account being deactivated
# here, we (conservatively) only concurrently invoke lambda function 5 times
data = [{"num_points": i} for i in range(1495, 1500)]
data # note 5 distinct tasks (mapping onto 5 distinct Lambda invocations)

In [ ]:
sfn_response = sfn.start_sync_execution(
    stateMachineArn=state_machine_arn,
    name='sync_test',
    input=json.dumps(data)
)

print(sfn_response['output'])

In [31]:
pi_sum, counter = 0, 0

for thread in json.loads(sfn_response['output']):
    pi_sum += thread['body']['pi_estimate']
    counter += 1

print(pi_sum/counter)

3.143235340481622
